In [17]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [18]:
class CDM:
    def __init__(self, h):
        self.h = h
    
    def diff(self, f, x):
        numerator = f(x + self.h) - f(x - self.h)
        denominator = 2 * self.h

        return numerator / denominator

In [19]:
class Newton:
    def __init__(self, f, CDM_object, tol=1e-6, max_iter=1000):
        self.f = f
        self.CDM = CDM_object
        self.tol = tol 
        self.max_iter = max_iter

    def solve(self, y, x0):
        x = x0
        for _ in range(self.max_iter):
            f_x = self.f(x) - y
            f_prime_x = self.CDM.diff(self.f, x)
            if abs(f_prime_x) < 1e-10:
                raise ValueError("Derivative is zero, method fails.")
            x_new = x - f_x / f_prime_x
            if abs(x_new - x) < self.tol:
                return x_new
            x = x_new

        raise ValueError(f"Method did not converge.({x_new})")

In [20]:
class LCG:
    def __init__(self, seed, a=561860773102413563, c=0, m=2**60-93):
        self.seed = seed
        self.a = a
        self.c = c
        self.m = m
        self.state = seed

    def next(self):
        self.state = (self.a * self.state + self.c) % self.m
        return self.state / self.m  # Normalize to [0, 1)
    
    # def next_in_range(self, a, b):
    #     return a + (b - a) * self.next()

In [21]:
import scipy.special
import numpy as np

In [22]:
if __name__ == '__main__':

    # ----------------------------------PART1----------------------------------

    def cdf(x): # F_X
        return 1/2 + 1/2 * \
            scipy.special.erf((np.log(x) - 2)/(np.sqrt(0.4)))

    cdm    = CDM(h=1e-6)
    newton = Newton(cdf, cdm, tol=1e-6, max_iter=1000)

    def inverse(y, x0): # x = f^-1(y)
        return newton.solve(y, x0)

    n = 120
    lcg = LCG(seed=42)

    # ----------------------------------PART2----------------------------------

    data = [lcg.next() for _ in range(n)]
    # print(data)

    guesses = [0, 3, 6, 9, 12, 15, 18, 21]
    for ind, el in enumerate(data):
        for attempt, guess in enumerate(guesses):
            try: 
                inv_value = inverse(el, guess)
                data[ind] = inv_value
                break
            except:
                pass

            if attempt == len(guesses) - 1:
                raise Exception('Solution was not found')
    
    # print(data)

    # ----------------------------------PART3----------------------------------

    mini, maxi = min(data), max(data)
    print(mini, maxi)

    range = maxi - mini
    print(range)

3.1657607190927446 36.348704412636216
33.18294369354347


<center> <h> TESTING </h> </center>

In [23]:
from scipy.stats import lognorm

mu = 2
sigma = np.sqrt(0.2)

lognorm_dist = lognorm(s=sigma, scale=np.exp(mu))

def foo(x):
    # return math.sin(x)
    # return x**3 - 2 * x - 5
    return lognorm_dist.cdf(x)

def inv_foo(y):
    return lognorm_dist.ppf(y)


In [24]:
if __name__ == '__main__':
    def cdf(x): # F_X
        return 1/2 + 1/2 * scipy.special.erf((np.log(x) - 2)/(np.sqrt(0.4)))

    for i in range(0, 15):
        print(f'{foo(i)}   {cdf(i)}')

    cdm    = CDM(h=1e-6)
    newton = Newton(cdf, cdm, tol=1e-6, max_iter=1000)

    def inverse(y, x0): # x = f^-1(y)
        return newton.solve(y, x0)

    print(f'')

    data    = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    guesses = [0, 3, 6, 9, 12, 15]

    for el in data:
        print(f'scipy: {inv_foo(el)}')
        for guess in guesses:
            try: 
                print(f'mine: {newton.solve(el, guess)}')
                break
            except:
                print(f'guess {guess} did not converge') 
        print('-'*200)

TypeError: 'numpy.float64' object is not callable